In [1]:
!pip install openai


In [12]:
!pip install pytesseract Pillow
## install for image bases text extraction

In [2]:
!pip install -q PyMuPDF
## install for pdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 80.3 MB/s eta 0:00:00


In [3]:
from openai import OpenAI

# Setup Groq API
client = OpenAI(
    api_key="gsk_...YOUR_KEY...",
    base_url="https://api.groq.com/openai/v1"
)


In [4]:
#upload a pdf file
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]


Saving sample_medical_report.pdf to sample_medical_report.pdf


In [28]:
#upload a image
from google.colab import files

uploaded = files.upload()
image_path = list(uploaded.keys())[0]


Saving medical.jpg to medical (1).jpg


In [16]:
#upload a tect file
from google.colab import files

uploaded = files.upload()  # Upload a plain text file (.txt)
txt_path = list(uploaded.keys())[0]

Saving Medical.txt to Medical.txt


In [8]:
#Extracting text from pdf
import fitz

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)  # Open the PDF
    text = ""
    for page in doc:    # Loop through each page
        text += page.get_text() # Extract text from that page
    doc.close() # Close the file
    return text.strip() # Return clean text

pdf_text = extract_text_from_pdf(pdf_path)

print(" Extracted Medical Text:\n")
print(pdf_text[:1000])
extracted_text = extract_text_from_pdf(pdf_path)


 Extracted Medical Text:

Patient Name: John Doe
Age: 58
Date: 2025-07-12
Chief Complaint:
Patient complains of shortness of breath, occasional chest pain, and fatigue over the past 2 weeks.
Clinical Findings:
- Heart rate: 110 bpm (tachycardia)
- Blood pressure: 145/90 mmHg
- Oxygen saturation: 94%
- ECG: Shows sinus tachycardia with occasional PVCs
- Echocardiogram: Mild left ventricular hypertrophy; ejection fraction of 45%
- Chest X-ray: Slight pulmonary congestion
Impression:
The findings are suggestive of mild congestive heart failure, likely secondary to chronic hypertension.
Plan:
Recommend lifestyle modification, initiate beta-blocker, low-sodium diet, and schedule a follow-up in 2 wee


In [30]:
#Extracting text from image
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract

def preprocess_and_ocr(image_path):
    # Load image
    img = Image.open(image_path)

    # Convert to grayscale
    img = img.convert("L")

    # Increase contrast
    enhancer = ImageEnhance.Contrast(img)
    img = enhancer.enhance(2)

    # Sharpen image
    img = img.filter(ImageFilter.SHARPEN)

    # Resize (double size for better OCR)
    img = img.resize((img.width * 2, img.height * 2))

    # Perform OCR
    text = pytesseract.image_to_string(img)
    return text.strip()
print(" Extracted Medical Text:\n")
print(extracted_text)
extracted_text = preprocess_and_ocr(image_path)

 Extracted Medical Text:

oo
a
\
K AMSNENI
——————————————————
HOSPITALS
A SS
oe
op Bill No BILL-OP- 103566 Bill Date - 30/08/2022 08:33AM Consultant : Dr. Swarajyam Veeramachaneni
M.D DNB
yliD » KHC-107736 Name - Mr. KONAKALA VENKATA Visit Date : 30/08/2022 08:34AM
KRISHNA RAO (45- Year(s)/M)
visit Type OP Phone No : 8801439972 Passport No:
visit Valid : 14-09-2022 Ref Con.

| Upto
Qty OP Consultation Fee

Dept/Docotor
Cardiology Consultation ] 100.0

 

 

VITALS: 2 be
PR: BE | Ay) RR BP; is D q () TEMP: O C}. § |
Nutritional Stays: OObese ONormalO MalNourished Ht: V eight: MI:
Dietician Consultation: D Required O Not Required Drug Allergies: OYES ONO / XE
Na
Nel prem ho Als
Mo vViLal ¥(lnem - ¥¢ covered
lag c ét OPT ‘
Candfa C agree eos usp fonn Oo bs
qt
coe nl
pas
. 9¢
pees ” 2b 0
K €@rneouec AM ype (f
an pau

 

ea deel =
ly Art ie et =e

    
 

Poranki, Vijayawada - 521 137, AP, India. « Ph.: 0866-246 3333, Fax: 0866-246 3300


In [22]:
#Extracting text from text file
with open(txt_path, "r") as file:
    txt_content = file.read().strip()

# Print preview
print("Extracted Medical Text:\n")
print(txt_content[:1000])
extracted_text = txt_content

Extracted Medical Text:

Medical Case Report
Patient ID: 345678
Name: Michael Johnson
Age: 29
Gender: Male
Date of Report: 2024-09-04

Chief Complaint:
The patient reports experiencing sudden episodes of intense chest pain, heart palpitations, shortness of breath, dizziness, and sweating over the past three months. These episodes typically last for about 10-20 minutes and occur without warning, usually once or twice a week. The patient describes a feeling of impending doom during these episodes and fears having a heart attack.

Medical History:
Family History: No known history of heart disease. Mother has generalized anxiety disorder; father has no significant medical history.
Personal Medical History:
Anxiety: Diagnosed at age 25; managed with cognitive behavioral therapy (CBT) and occasional use of benzodiazepines.
Gastroesophageal Reflux Disease (GERD): Diagnosed at age 27; managed with proton pump inhibitors (PPIs) and dietary changes.
Lifestyle Factors: The patient works in a high

In [9]:
#Simplifies report for pdf file
prompt = f"""
You're a helpful medical assistant.
Simplify the following medical report for someone with no medical background.
Avoid medical jargon. Use clear, everyday language.

Medical Report:
{extracted_text}
"""

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

print("Simplified Medical Report:\n")
print(response.choices[0].message.content)

Simplified Medical Report:

Here's a simplified version of the medical report:

**John Doe's Medical Report**

**What's Going On:**
John has been feeling short of breath, having some chest pain, and feeling really tired over the past two weeks.

**What the Doctor Found:**
When John came in for a check-up, the doctor took some measurements and did some tests. Here's what they found:

* His heart is beating a bit too fast (110 times per minute).
* His blood pressure is a bit high (145/90).
* His oxygen levels are okay, but not perfect (94%).
* The doctor did a special test to look at John's heart (ECG) and found that it's beating a bit irregularly.
* Another test (echocardiogram) showed that John's heart is a bit thicker than it should be, and it's not pumping blood as well as it should (45%).
* An X-ray of John's chest showed that there's a bit of fluid buildup in his lungs.

**What It Means:**
All of these findings suggest that John has a mild case of heart failure, which is likely cau

In [31]:
#Simplified report for image
prompt = f"""
You're a helpful medical assistant.
Simplify the following medical report for someone with no medical background.
Avoid medical jargon. Use clear, everyday language.

Medical Report:
{extracted_text}
"""

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

print("Simplified Medical Report:\n")
print(response.choices[0].message.content)

Simplified Medical Report:

Here's a simplified version of the medical report:

**Patient Information:**

* Name: Mr. Konakala Venkata Krishna Rao
* Age: 45 years old
* Male

**Visit Details:**

* Date: August 30, 2022
* Time: 8:34 AM
* Doctor: Dr. Swarajyam Veeramachaneni (Cardiology Consultant)
* Hospital: Not specified (but the contact details are from Vijayawada, AP, India)

**Medical Information:**

* The patient had a consultation with the doctor in the cardiology department.
* The doctor checked the patient's vital signs, which are:
	+ Pulse rate (PR): normal
	+ Respiratory rate (RR): normal
	+ Blood pressure (BP): normal
	+ Body temperature (TEMP): normal
* The patient's height and weight are recorded, but not specified.
* The doctor did not find any signs of malnutrition.
* The patient does not have any known drug allergies.

**Additional Notes:**

* The patient may need to see a dietician (a specialist who advises on healthy eating) in the future.
* The patient has been advis

In [23]:
#Simplified report for txt file
prompt = f"""
You're a helpful medical assistant.
Simplify the following medical report for someone with no medical background.
Avoid medical jargon. Use clear, everyday language.

Medical Report:
{extracted_text}
"""

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3
)

print("Simplified Medical Report:\n")
print(response.choices[0].message.content)

Simplified Medical Report:

Here's a simplified version of the medical report:

**Summary of Michael Johnson's Medical Report**

Michael Johnson, a 29-year-old man, has been experiencing scary symptoms over the past three months. He gets sudden chest pain, a racing heart, shortness of breath, dizziness, and sweating. These episodes usually last 10-20 minutes and happen without warning, about once or twice a week. Michael is worried that he might be having a heart attack.

**Medical Background**

Michael has a history of anxiety, which he's been managing with therapy and occasional medication. He also has acid reflux, which is controlled with medication and diet changes. His family doesn't have a history of heart disease, but his mom has anxiety. Michael works a high-stress job, occasionally drinks caffeine and alcohol, and doesn't exercise regularly.

**Test Results**

The doctors did several tests to check Michael's heart and overall health. The good news is that:

* His heart rhythm 